<a href="https://colab.research.google.com/github/restrepo/BSM-Submodules/blob/master/SARAH-SSDM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SARAH configuration files generator

In [1]:
import os
if os.getcwd()=='/content':
    !wget -O SARAH.py https://raw.githubusercontent.com/restrepo/BSM-Submodules/master/SARAH.py 2>/dev/null
    !wget -O cmdlike.py https://raw.githubusercontent.com/restrepo/BSM-Submodules/master/cmdlike.py 2>/dev/null
    !git clone https://github.com/restrepo/SARAH.git 2>/dev/null
    !mkdir -p JSON
    os.chdir('JSON')
    !wget -O fullparticles.json https://raw.githubusercontent.com/restrepo/BSM-Submodules/master/JSON/fullparticles.json 2> /dev/null
    !wget -O fullparticlesnames.json https://raw.githubusercontent.com/restrepo/BSM-Submodules/master/JSON/fullparticlesnames.json 2> /dev/null
    !wget -O fullparametersnames.json https://raw.githubusercontent.com/restrepo/BSM-Submodules/master/JSON/fullparametersnames.json 2> /dev/null
    os.chdir('../')

In [2]:
import pandas as pd
import json
import re
import numpy as np
import sys
from SARAH import *
pd.set_option('display.max_colwidth',200)

## Choose step:

In [3]:
PARTICLES =False
PARAMETERS=True
SPHENO    =False
OUTPUT    =False
BSM=False
if PARAMETERS:
    PARTICLES=False
if SPHENO:
    PARAMETERS=False
    PARTICLES=False
if OUTPUT:
    SPHENO=False
    PARAMETERS=False
    PARTICLES=False    
if not PARTICLES and not PARAMETERS and not SPHENO and not OUTPUT:
    print('SETTING BSM True')
    BSM=True
if BSM:    
    OUTPUT=False
    SPHENO=False
    PARAMETERS=False
    PARTICLES=False
PARTICLES,PARAMETERS,SPHENO,OUTPUT,BSM

(False, True, False, False, False)

## Read `MODEL.m` file into an standarized list of dictionaries 

### Design
#### Class inheritance
* Basic object
```
list_of_list_of_dictionaries -> Particles -> Parameters
```
* Methods
```
SARAH -> update_SARAH_particles -> update_SARAH_parameters -> update_SARAH_SPheno -> update_SARAH
```


#### USAGE:
1. Load a `SARAH` Model File
2. Warns if a `particle` or `parameter` is not yet defined
3. Build missing `particles` or `parameters` and update predefined particles loaded in `__init__`
4. Export `SARAH` auxiliarly files:
   * `particles.m`   
   * `parameters.m`
   * `SPheno.m`

In [4]:
from SARAH import *

## Update particles

### How to build a function for the `self.update_particles` method
For one Specific Type of Particles: `self.update_particles(f,Definition,Lorentz,args)` is expected to filter the set of particles
by using the  `Definition`  and `Lorentz` arguments, and pass the resulting list of particles (as an object `Particles`) to function `f(particle,*args)`, where `particle` is each one of the elements of the filtered list of particles.  The function must return another `Particles` object with the help of the extra arguments passed through the optional tuple: `args=(...)`

The main purpose of the generic function `f` is either to 
1. add additional info to an existing particle: Define a function with a mandatory `particle` argument which returns the  same particle with additional info like `p['Description]`. The recommended name for the function in such a case is `get_Specific_Type_of_Particles_Descriptions(particle,...)`
1. generate a new non-existing particle. In such a case the recommended name for the function is `get_Specific_Type_of_Particles(particle,...)`. Each existing particle  must generare one and only one new particle

__Example__:

Define
```python
def get_WeylFermion_Description(p,particles):
    p.get('Properties')['update_Description']={'LaTeX' : get_weylfermion_LaTeX( p.get('Field') )}
    return p
```
and use
```python
self.update_particles( get_WeylFermion_Descriptions, 
                      Definition='WeylFermionAndIndermediate',
                      Lorentz='WeylFermion')
```

In [5]:
def get_4Spinor_Descriptions(particle,particles):
    """
    Function which that take the dictionary particle and adds: 
    'Description' or 'Properties' -> 'update_Description'
    To the particle with 'Field' some 4Spinor
    """
    p=particle
    gprt=particles.loc[ p.get('Parents') ]
    if gprt[0].get('rotation'):
        gprt=particles.loc[gprt[0].get('Parents')]
    cmpt=gprt[0].get('Field')
    gprt=particles.loc[gprt[0].get('Parents')]
    if get_abs_groups( gprt[0].get('Properties').get('Groups'), u1_abs='1/6',su2='2',su3_abs='3' ):
        if cmpt==gprt[0].get('Properties').get('multiplet')[0]:
            p['Description']='Up-Quarks'
        else:
            p['Description']='Down-Quarks'
    if get_abs_groups( gprt[0].get('Properties').get('Groups'), u1_abs='1/2',su2='2',su3_abs='1' ):
        if cmpt==gprt[0].get('Properties').get('multiplet')[0]:
            p['Description']='Neutrinos'
        else:
            p['Description']='Leptons'
    return p

def get_WeylFermion_Descriptions(p):
    '''
    All the Weyl Fermions are used. To filter simplified iso-singlet names, use:
        if not p.get('Parents') and not p.get('Properties').get('multiplet'):
            RETURN=False
        #Empty dic: {}, must be returned when RETURN is False
    '''
    p.get('Properties')['update_Description']={'LaTeX' : get_weylfermion_LaTeX( p.get('Field') )}
    return p

def get_gauge_vectors_Descriptions(p,particles,
                di={'U[1]' :{'Description':'B-Boson' ,'Ghost':'B-Boson Ghost' },
                    'SU[2]':{'Description':'W-Bosons','Ghost':'W-Boson Ghost'},
                    'SU[3]':{'Description':'Gluon'   ,'Ghost':'Gluon Ghost'}}):
    gr=p.get('Properties').get('Group').strip()
    if di.get( gr  ):
        p['Description']=di.get(gr).get('Description')
    return p

def get_ghosts(p,particles,
                di={'U[1]' :{'Description':'B-Boson' ,'Ghost':'B-Boson Ghost' },
                    'SU[2]':{'Description':'W-Bosons','Ghost':'W-Boson Ghost'},
                    'SU[3]':{'Description':'Gluon'   ,'Ghost':'Gluon Ghost'}}):
    gr=p.get('Properties').get('Group').strip()
    if di.get( gr  ):
        p=particles.apply_filter(lambda d: d.get('Description')==di.get(gr).get('Ghost'))[0]
        return p
    else:
        return {}
    
def get_boson_vectors_Descriptions(p,particles,partialparticles,
                      di={'B-Boson':'Photon',
                          'W-Bosons':{1:'W+ - Boson',3:'Z-Boson'}},
                      goldston={
                          'W-Bosons':{1:'Charged Higgs',3:'Pseudo-Scalar Higgs'}
                                      }):
    '''
    Assumptions:
    1) The Goldston bosons are the SM ones and this function
    must be called after the scalar definitions
    '''
    pP=p.get('Parents')
    if not pP:
        pP=''
    abelian=pP.split('[')
    try:
        BV=particles.loc[abelian[0]]
        dscr=BV[0].get('Description')
    except (IndexError,KeyError):
        dscr=''
        
    la=len(abelian)
    gltn=Particles([])
    if la==1:
        get_dscrp=di.get(dscr)
    else:
        try:
            i=eval( re.search('\[\s*([0-9]+)\s*\]',pP).groups()[0] )
        except (IndexError,AttributeError):
            i=-1
            
        try:
            get_dscrp=di.get(dscr).get(i)
            gltn=partialparticles.apply_filter(
                lambda d: d.get('Description')==goldston.get(dscr).get(i) 
                     )
        except AttributeError:
            get_dscrp=''
            gltn=Particles([])
            
    if get_dscrp:
        p['Description']=get_dscrp
        
    if gltn.size()==1:
        p['Properties']['update_Description']={'Goldstone':gltn.get('Field')[0]}        
        

    return p

def get_boson_vectors_ghosts(p,particles,
                      di={'B-Boson':'Photon Ghost',
                          'W-Bosons':{1:'Positive W+ - Boson Ghost',
                                      3:'Z-Boson Ghost'}}):
    np={}
    pP=p.get('Parents')
    if not pP:
        pP=''
    abelian=pP.split('[')
    try:
        BV=particles.loc[abelian[0]]
        dscr=BV[0].get('Description')
    except (IndexError,KeyError):
        dscr=''    
    la=len(abelian)
    gltn=Particles([])
    if la==1:
        get_dscrp=di.get(dscr)
    else:
        try:
            i=eval( re.search('\[\s*([0-9]+)\s*\]',pP).groups()[0] )
        except (IndexError,AttributeError):
            i=-1
            
        get_dscrp=di.get(dscr).get(i)

    if get_dscrp:
        np['Field']      =re.sub('^V','g',  p.get('Field'))
        np['Description']=get_dscrp
        np['Block']      = 'GaugeSector'
        np['Definition'] = 'EWSB'
        np['Properties'] ={'Lorentz': 'Ghost'}
        return np
    else:
        return {}

def get_IntermediateScalars_Definitions(p):
    if get_abs_groups( 
         p.get('Properties').get('Groups'),u1_abs='1/2',su2='2',su3_abs='1'):
        if p.get('Properties').get('Groups')[0].find('-')==-1:
            p.get('Properties')['charge']=['Charged','Neutral']
        else:
            p.get('Properties')['charge']=['Neutral','Charged']

    p.get('Properties')['update_Description']={
                                               'PDG'       : [0],
                                               'Width'     : 0,
                                               'Mass'      : 'Automatic',
                                               'LaTeX'     : p.get('Field'),
                                               'OutputName': p.get('Field') }
    return p

def get_GaugeEScalars_Definitions(p,H):
    if H.size()==1 and p.get('Parents')==H.get('Field')[0]:
        #Standard model Higgs
        p.get('Properties')['update_Description']={'PDG'  :[0],
                                               'Width':0,
                                               'Mass' : 'Automatic',
                                               'OutputName' : p.get('Field')
                                               }

        l=H[0].get('Properties').get('multiplet')
        Hp0=l.index(p.get('Field'))
        if H[0].get('Properties').get('charge')[Hp0]=='Charged':
            p.get('Properties')['ElectricCharge']=1
            p.get('Properties')['update_Description']['FeynArtsNr']=2
            p.get('Properties')['update_Description']['LaTeX']='H^+'
        elif H[0].get('Properties').get('charge')[Hp0]=='Neutral':
            p.get('Properties')['ElectricCharge']=0
            p.get('Properties')['update_Description']['FeynArtsNr']=1
            p.get('Properties')['update_Description']['LaTeX']='H^0'
    return p

def get_EWSBScalars_Definitions(p,particles):
    if p.get('Properties').get('CP')=='Real':
        pp=particles.apply_filter(lambda d: d.get('Description')=='Higgs')
        try:
            pphh=pp[0]
        except IndexError:
            pphh={}
        if pphh:
            p['Description']=pphh.get('Description')
            p.get('Properties')['update_Description']=pphh.get('Properties').get(
                                                            'update_Description')
    elif p.get('Properties').get('CP')=='Imaginary':
        pp=particles.apply_filter(lambda d: d.get('Description')==
                                                        'Pseudo-Scalar Higgs')
        try:
            ppA0=pp[0]
        except IndexError:
            ppA0={}
        if ppA0:
            p['Description']=ppA0.get('Description')
            p.get('Properties')['update_Description']=ppA0.get('Properties').get(
                                                            'update_Description')
        
    return p    

def get_EWSBScalars(p,H,particles):
    '''
    Only Works if 
    H=self.update_particles(get_IntermediateScalars_Definitions,
                               Definition='WeylFermionAndIndermediate',
                               Lorentz='Scalar').size()==1
    '''
    pp={}
    if H.size()==1 and p.get('Field')==H.get('Field')[0]:
        kk=particles.apply_filter(lambda d: d.get('Name')=='Charged Higgs')
        pp=kk[0]
        Hp=H[0].get('Properties').get('charge').index('Charged')
        try:
            Hpname=H[0].get('Properties').get('multiplet')[Hp]
            pp['Field']  = Hpname
            pp['Parents']= Hpname
        except IndexError:
            pass
    return pp

def fix_gluon(d):
    if d.get('Description')=='Gluon':
        d['Definition']='EWSB'
    if d.get('Description')=='Gluon Ghost':
        d['Definition']='EWSB'
    return d

In [6]:
class update_SARAH_particles(SARAH):
    def __init__(self,*args, **kwargs):
        super().__init__(*args, **kwargs)

    def update_fermions(self):
        NP=Particles([])
        NP=self.update_particles(get_4Spinor_Descriptions,
                                Definition='EWSB',
                                Lorentz='DiracSpinor',
                                args=(self.modelparticles))
        NP=NP+self.update_particles(get_4Spinor_Descriptions,
              Definition='EWSB',Lorentz='MajoranaSpinor',args=(self.modelparticles))
        NP=NP+self.update_particles( get_WeylFermion_Descriptions, 
                                Definition='WeylFermionAndIndermediate',
                                Lorentz='WeylFermion')
        return Particles(NP)

    def update_gauge_bosons(self):
        NP=Particles([])
        NP=NP+self.update_particles(get_gauge_vectors_Descriptions,
                                Definition='GaugeES',
                                Lorentz='Vector',
                                args=(self.modelparticles,
                            {'U[1]'  :{'Description':'B-Boson' ,'Ghost':'B-Boson Ghost'},
                             'SU[2]' :{'Description':'W-Bosons','Ghost':'W-Boson Ghost'},
                             'SU[3]' :{'Description':'Gluon'   ,'Ghost':'Gluon Ghost'}}))
        NP=NP+self.update_particles(get_ghosts,
                                Definition='GaugeES',
                                Lorentz='Vector',
                                args=(self.particles,
                            {'U[1]'  :{'Description':'B-Boson' ,'Ghost':'B-Boson Ghost'},
                             'SU[2]' :{'Description':'W-Bosons','Ghost':'W-Boson Ghost'},
                             'SU[3]' :{'Description':'Gluon'   ,'Ghost':'Gluon Ghost'}}))
        NP=NP+self.update_particles(get_boson_vectors_Descriptions,
                                Definition='EWSB',
                                Lorentz='Vector',
                                args=(NP,
                                      self.updated_modelparticles,
                            {'B-Boson':'Photon',
                             'W-Bosons':{1:'W+ - Boson',3:'Z-Boson'}}))
        NP=NP+self.update_particles(get_boson_vectors_ghosts,
                                Definition='EWSB',
                                Lorentz='Vector',
                                args=(NP,
                            {'B-Boson':'Photon Ghost',
                             'W-Bosons':{1:'Positive W+ - Boson Ghost',
                                         3:'Z-Boson Ghost'}}))
        #Special case: Negative ghosts
        negative_ghost='Negative W+ - Boson Ghost'
        NP=NP+self.update_particles(get_boson_vectors_ghosts,
                                Definition='EWSB',
                                Lorentz='Vector',
                                args=(NP,
                            {'B-Boson':'',
                             'W-Bosons':{1:negative_ghost,
                                         3:''}})).apply_filter(
                            lambda d: d.get('Description')==negative_ghost
                                                      ).apply(fC)
        
        #Special case: copy GaugeES Gluon (Ghost) to  EWSB
        #NP=Particles(NP.apply(fix_gluon))
        G=[]
        for d in NP.apply_filter(lambda d: d.get('Description').find('Gluon')>-1):
            dd=d.copy()
            dd['Definition']='EWSB'
            G.append(dd )
        G=Particles(G)
        NP=NP+G
        
        return Particles(NP)

    def update_scalars(self):
        '''
        The order matters!
        '''
        NP=Particles([])
        
        H=self.update_particles(get_IntermediateScalars_Definitions,
                                    Definition='WeylFermionAndIndermediate',
                                    Lorentz='Scalar')
        NP=NP+H
        
        SMVEV=False
        VEVs=self.modelparticles.apply_filter(
            lambda d: d.get('Block')=='VEVs' )
        if VEVs.size()==2:
            SMVEV=True
        
        #SM-like 'WeylFermionAndIndermediate' (WFI) states, `HSM` + `INERTs` WFI states
        if SMVEV:
            HSM_name=self.modelparticles.loc[VEVs.get('Parents')[0]].get('Parents')[0]    
            HSM=H.loc[HSM_name]
            INERTs=H.apply_filter( lambda d: d.get('Field')!=HSM_name)
        #TODO: Multi-vev potentials
        else:
            HSM=H
            INERTs=Particles([])        

        #Process EWSB standard model particles    
        NP=NP+self.update_particles(get_EWSBScalars_Definitions,
                                    Definition='EWSB',
                                    Lorentz='Scalar',args=(self.particles))
        
            
        #For SM-like Fields Descriptions added,
        HpH0=self.update_particles(get_GaugeEScalars_Definitions,Definition='GaugeES',
                                    Lorentz='Scalar',
                                    args=(HSM))
        NP=NP+HpH0

        #Repeat 'WeylFermionAndIndermediate' Hp goldstone-field as EWSB field
        # to accomodate weird SARAH design
        kk=self.update_particles(get_EWSBScalars,
                                    Definition='WeylFermionAndIndermediate',                    
                                    Lorentz='Scalar',args=(HSM,self.particles))
        kk=kk.apply_filter(lambda d: isinstance(d.get('Field'),str))

        NP=NP+kk
        
        #non 'WeylFermionAndIndermediate' Beyond  SM scalar field

        return Particles(NP)
            
    def update_modelparticles(self):            
        #must be first
        self.updated_modelparticles=self.update_scalars()

        # GaugeBosons
        self.updated_modelparticles=self.updated_modelparticles+self.update_gauge_bosons()

        #Scalars
        self.updated_modelparticles=Particles(self.updated_modelparticles+
                                              self.update_fermions() )

        return self.updated_modelparticles

In [7]:
if PARTICLES:
    us=update_SARAH_particles(model='SSDM')
    #us=update_SARAH_particles(model='SM')
    #newp=us.parse_model_particles()
    kk=us.update_modelparticles()
    self=us

In [8]:
us=update_SARAH_particles(model='SSDM')
    #us=update_SARAH_particles(model='SM')
    #newp=us.parse_model_particles()
kk=us.update_modelparticles()
self=us

## Parse and update parameters

In [9]:
class update_SARAH_parameters(update_SARAH_particles):
    def __init__(self,*args, **kwargs):
        super().__init__(*args, **kwargs)
    
    def Lagrangian_Couplings_Descriptions(self):
        '''
        '''
        dd=self.Lagrangian
        smdict=self.Lagrangian_Couplings
        particles=self.updated_modelparticles

        for k in dd.keys():
            if (sorted_equality( smdict['Up-Yukawa-Coupling']['Lorentz'],dd[k]['Lorentz'] ) and
                sorted_equality( smdict['Up-Yukawa-Coupling']['hypercharge'],dd[k]['hypercharge'] ) ):
                #Get Higgs from scalar part
                for i in range(len(dd[k]['Lorentz'])):
                    if dd[k]['Lorentz'][i]=='Scalar':
                        H=dd[k]['fields'][i]
                        smdict['Up-Yukawa-Coupling']['Coupling']=k
                        smdict['Up-Yukawa-Coupling']['Higgs']=H
                        smdict['Up-Yukawa-Coupling']['fields']=dd[k]['fields']
                #Use obtained Higgs to obtain diagonal form
                if not H:
                    sys.exit('Higgs doublet field symbol not found!')
                for i in range(len(dd[k]['Lorentz'])):                
                    if dd[k]['Lorentz'][i]=='WeylFermion':
                        mltp=get_multiplet(dd[k]['fields'][i],particles)
                        for p in mltp.keys():
                            if mltp[p].get('pos')=='Up':
                                vev=get_higgs_vev(H,particles)
                                smdict['Up-Yukawa-Coupling'
                                      ]['update_Description'
                                      ]['DependenceNum']=get_diagonal_basis(
                                                         vev,p,particles)                

        if not smdict.get('Up-Yukawa-Coupling'):
            sys.exit('"Up-Yukawa-Coupling" NOT FOUND!' )
        lk=list(dd.keys())
        try:
            lk.remove( smdict['Up-Yukawa-Coupling']['Coupling'] )
        except KeyError:
            pass

        lds=list(smdict.keys())
        lds.remove('Up-Yukawa-Coupling')
        for ds in lds:
            for k in lk:
                if smdict[ds].get('Lorentz'
                    ) and (sorted_equality( smdict[ds]['Lorentz'],dd[k]['Lorentz'] ) and
                    sorted_equality( smdict[ds]['hypercharge'],dd[k]['hypercharge'] ) ):
                    smdict[ds]['Coupling']=k
                    smdict[ds]['fields']=dd[k]['fields']
                    #Fix Descriptions for Higgs mass potential parameter
                    if ds=='SM Mu Parameter':
                        if not smdict[ds].get('update_Description'):
                            smdict[ds]['update_Description']={}
                        smdict[ds]['update_Description']['OutputName']=smdict[ds]['Coupling']
                        smdict[ds]['update_Description']['LaTeX']     =symbol_to_TeX(
                                                                       smdict[ds]['Coupling'])
                    #'SM Higgs Selfcouplings'                        
                    if H:
                        smdict[ds]['Higgs']=H
                    for i in range(len(dd[k]['Lorentz'])):
                        if dd[k]['Lorentz'][i]=='WeylFermion':
                            mltp=get_multiplet(dd[k]['fields'][i],particles)
                            for p in mltp.keys():
                                if mltp[p].get('pos')=='Down':
                                    Hk=[dd[k].get('fields')[i] for i in range(len( dd[k]['Lorentz'] ))
                                         if dd[k]['Lorentz'][i]=='Scalar' ][0]
                                    vev=get_higgs_vev(Hk,particles)
                                    smdict[ds]['update_Description'
                                          ]['DependenceNum']=get_diagonal_basis(
                                                             vev,p,particles)
        self.Lagrangian_Couplings=smdict
        self.Higgs=H
        return smdict
            
    def get_rotations(self):
        rotation={}
        smdict=self.Lagrangian_Couplings
        particles=self.updated_modelparticles
        for k in smdict.keys():
            try:
                fields=range(len(smdict[k].get('fields')))
            except TypeError:
                fields=[]
            for i in fields:
                if smdict[k]['Lorentz'][i]=='WeylFermion':
                    mltp=particles.apply_filter(lambda d: d.get('Parents')==smdict[k]['fields'][i])
                    if mltp.size()==2:
                        chiral='Left'
                    elif mltp.size()==1:
                        chiral='Right'
                    else:
                        chiral=None

                    j=0
                    for p in mltp.get('Field'):
                        j=j+1
                        prt=particles.loc[p]
                        rot=particles.apply_filter(
                            lambda d: str(d.get('Parents')).find(p)>-1
                                                 ).get('rotation')[0]
                        if isinstance(rot,str):
                            rotation[p]={rot:{}}
                            dscr='Mixing-Matrix'
                            if chiral=='Left':
                                if j==1:
                                    dscr='{}-Up-{}'.format(chiral,dscr)
                                elif j==2:
                                    if re.search('^[dD]',p):
                                        dscr='{}-Down-{}'.format(chiral,dscr)
                                    elif re.search('^[eE]',p):
                                        dscr='{}-Lepton-{}'.format(chiral,dscr)


                            elif chiral=='Right':
                                if re.search('^[uU]',p):
                                    dscr='{}-Up-{}'.format(chiral,dscr)
                                elif re.search('^[dD]',p):
                                    dscr='{}-Down-{}'.format(chiral,dscr)
                                elif re.search('^[eE]',p):
                                    dscr='{}-Lepton-{}'.format(chiral,dscr)
                                else:
                                    dscr=None
                            rotation[p][rot]['Description']=dscr

        self.rotations=rotation
        return rotation
    
    def get_couplings(self):
        particles=self.updated_modelparticles
        rotation=self.rotations

        prtng=particles.apply_filter(
                lambda d: str(d.get('Description')).lower().find('ghost')==-1
                     )
        grps=prtng.apply_filter(
                lambda d: d.get('Properties').get('Group')!=None
                     ).apply(
                lambda d: d.get('Properties').get('Group')
                )

        G={}
        coupling={}
        for g in grps:
            G=prtng.apply_filter(lambda d: d.get('Properties').get('Group')==g)
            V=G[0].get('Field')
            try:
                VV=prtng.apply_filter(lambda d: str(d.get('Parents')).find(V)>-1)
                # Non-Abelian case with SSB
                if VV.size()==2:
                    key='Properties'
                    pattern='conj\[\w+\]'
                    VV=VV.apply_filter( lambda d: re.search( pattern, str(d.get(key))  ) )

                f=VV[0]['Field']
                r=VV[0]['rotation']
            except IndexError:
                f=''  
            c=G[0].get('Properties').get('Coupling').strip()
            if g.find('U[1]')>-1:
                if f:
                    rotation[f]={r: {'Description':"Photon-Z Mixing Matrix"}}
                coupling[c]={'Description':'Hypercharge-Coupling'}
            if g.find('SU[2]')>-1:
                if f:
                    rotation[f]={r: {'Description':"W Mixing Matrix",
                                      'update_Description':
                                            {'Dependence' :r'''1/Sqrt[2] {{1, 1},
                                      {\[ImaginaryI],-\[ImaginaryI]}}''' }}}
                coupling[c]={'Description':'Left-Coupling'}
            if g.find('SU[3]')>-1:
                coupling[c]={'Description':'Strong-Coupling'}
        self.couplings=coupling
        return coupling
    
    def get_constants(self):
        self.constants={}
        self.constants['AlphaS']= { 'Description'  : 'Alpha Strong'}
        self.constants['e']     = { 'Description'  :  'electric charge'} 
        self.constants['Gf']    = { 'Description'  :  "Fermi's constant"}
        self.constants['aEWinv']= { 'Description'  :  'inverse weak coupling constant at mZ'}
        return self.constants
    
    def update_Lagrangian(self):
        particles=self.updated_modelparticles
        rotation=self.rotations
        coupling=self.couplings
        H=self.Lagrangian_Couplings['SM Higgs Selfcouplings'
                                   ].get('Higgs')
        if H:
            H0=get_H0(H,particles)
            if H0.size()>0:
                hh=get_hh(H0,particles).get('Field')[0]
                vev=get_hh(H0,particles).get('Properties')[0].get('vev')
                if hh and vev:
                    self.Lagrangian_Couplings['SM Higgs Selfcouplings'
                          ]['update_Description']={
                            'LaTeX': symbol_to_TeX(
                                      self.Lagrangian_Couplings[
                                          'SM Higgs Selfcouplings'
                                           ].get('Coupling'),script='_' ),
                            'OutputName': symbol_to_OutputName(
                                      self.Lagrangian_Couplings[
                                          'SM Higgs Selfcouplings'
                                           ].get('Coupling') ),
                            'DependenceNum': 
                                r'Mass[{}]^2/({}^2)'.format(hh,vev) }

        self.Lagrangian_Couplings['EW-VEV']={}
        self.Lagrangian_Couplings['EW-VEV']['Coupling']=vev
        self.Lagrangian_Couplings['EW-VEV']['update_Description']={}
        self.Lagrangian_Couplings['EW-VEV']['update_Description']['DependenceSPheno']=None
        self.Lagrangian_Couplings['EW-VEV']['update_Description']['OutputName']='vvSM'
        VWp=''
        g2=''
        for k in rotation.keys():
            if list( rotation[k].values() )[0].get('Description')=='W Mixing Matrix':
                VWp=k
        for c in coupling.keys():
            if coupling[c].get('Description')=='Left-Coupling':
                g2=c
        self.Lagrangian_Couplings['EW-VEV'
                                 ]['update_Description'
                                  ]['DependenceNum']=r'Sqrt[4*Mass[{}]^2/({}^2)]'.format(VWp,g2)

        for k in rotation:
            if list( rotation[k].values() )[0].get('Description')=='Photon-Z Mixing Matrix':
                VP=k
                ZZ=list(rotation[k].keys())[0]
        fz=particles.apply_filter( lambda d: d.get('rotation')==ZZ )
        VZ=fz.apply_filter(lambda d: d.get('Field')!=VP)[0]['Field']
        self.Lagrangian_Couplings['Weinberg-Angle']={}
        self.Lagrangian_Couplings['Weinberg-Angle']['Coupling']='ThetaW'
        self.Lagrangian_Couplings['Weinberg-Angle']['update_Description']={}
        self.Lagrangian_Couplings['Weinberg-Angle']['update_Description'
             ]['DependenceNum']='ArcSin[Sqrt[1 - Mass[{}]^2/Mass[{}]^2]]'.format(
                                                                   VWp,VZ )
        
    def update_modelparameters(self):
        smdict=self.Lagrangian_Couplings
        rotation=self.rotations
        coupling=self.couplings
        constants=self.constants
        parameters=[]

        parameters=append_Lagrangian_to_parameters(smdict,parameters)
        
        #rotation will be altered
        rt=copy.deepcopy(rotation)
        for k in rt.keys():
            for r in rt[k].keys():
                d={}
                d['Symbol']=r
                d['Description']=rt[k][r].get('Description')
                d['Name']=d['Description']
                d['Class']='Rotation'
                kk=rt[k][r].pop('Description')
                if rt[k][r]:
                    d['Properties']=rt[k][r]
                else:
                    d['Properties']={}
                parameters.append(d)
        del(rt)
        
        for k in coupling.keys():
            d={}
            d['Symbol']=k
            d['Description']=coupling[k].get('Description')
            d['Name']=d['Description']
            d['Class']='Coupling'
            d['Properties']={}
            parameters.append(d)
            
        for k in constants.keys():
            d={}
            d['Symbol']=k
            d['Description']=constants[k].get('Description')
            d['Name']=d['Description']
            d['Class']='Constant'
            d['Properties']={}
            parameters.append(d)
            
        parameters=Parameters(parameters,index='Symbol')
        self.updated_modelparameters=parameters
        return parameters        

    # Parse parameters
    def parse_model_parameters(self):
        #2)
        kk=self.update_modelparticles()
        kk=self.parse_Lagrangian()
        kk=self.Lagrangian_Couplings_Descriptions()
        kk=self.get_rotations()
        kk=self.get_couplings()
        kk=self.get_constants()
        kk=self.update_Lagrangian()
        kk=self.update_modelparameters()
        return kk

In [10]:
kk=self.update_modelparticles()
kk=self.parse_Lagrangian() #Fill self.Lagrangian

In [14]:
self.Lagrangian

{'LamS': {'Lorentz': ['Scalar', 'Scalar', 'Scalar', 'Scalar'],
  'fields': ['S', 'S', 'S', 'S'],
  'hypercharge': ['0', '0', '0', '0'],
  'operator': 'S.S.S.S'},
 'LamSH': {'Lorentz': ['Scalar', 'Scalar', 'Scalar', 'Scalar'],
  'fields': ['S', 'S', 'H', 'H'],
  'hypercharge': ['0', '0', '1/2', '1/2'],
  'operator': 'S.S.conj[H].H'},
 'Lambda1': {'Lorentz': ['Scalar', 'Scalar', 'Scalar', 'Scalar'],
  'fields': ['H', 'H', 'H', 'H'],
  'hypercharge': ['1/2', '1/2', '1/2', '1/2'],
  'operator': 'conj[H].H.conj[H].H'},
 'MS2': {'Lorentz': ['Scalar', 'Scalar'],
  'fields': ['S', 'S'],
  'hypercharge': ['0', '0'],
  'operator': 'S.S'},
 'Yd': {'Lorentz': ['Scalar', 'WeylFermion', 'WeylFermion'],
  'fields': ['H', 'd', 'q'],
  'hypercharge': ['1/2', '1/3', '1/6'],
  'operator': 'conj[H].d.q'},
 'Ye': {'Lorentz': ['Scalar', 'WeylFermion', 'WeylFermion'],
  'fields': ['H', 'e', 'l'],
  'hypercharge': ['1', '1/2', '1/2'],
  'operator': 'conj[H].e.l'},
 'Yu': {'Lorentz': ['Scalar', 'WeylFermion', 

In [15]:
self.Lagrangian_Couplings

{'Down-Yukawa-Coupling': {'Lorentz': ['Scalar', 'WeylFermion', 'WeylFermion'],
  'hypercharge': ['1/2', '1/3', '1/6'],
  'update_Description': {}},
 'Lepton-Yukawa-Coupling': {'Lorentz': ['Scalar',
   'WeylFermion',
   'WeylFermion'],
  'hypercharge': ['1', '1/2', '1/2'],
  'update_Description': {}},
 'SM Higgs Selfcouplings': {'Lorentz': ['Scalar',
   'Scalar',
   'Scalar',
   'Scalar'],
  'hypercharge': ['1/2', '1/2', '1/2', '1/2'],
  'update_Description': {}},
 'SM Mu Parameter': {'Lorentz': ['Scalar', 'Scalar'],
  'hypercharge': ['1/2', '1/2'],
  'update_Description': {'OutputName': 'm2SM'}},
 'Up-Yukawa-Coupling': {'Lorentz': ['WeylFermion', 'WeylFermion', 'Scalar'],
  'hypercharge': ['1/2', '1/6', '2/3'],
  'update_Description': {}}}

In [16]:
self.Lagrangian_Couplings

{'Down-Yukawa-Coupling': {'Lorentz': ['Scalar', 'WeylFermion', 'WeylFermion'],
  'hypercharge': ['1/2', '1/3', '1/6'],
  'update_Description': {}},
 'Lepton-Yukawa-Coupling': {'Lorentz': ['Scalar',
   'WeylFermion',
   'WeylFermion'],
  'hypercharge': ['1', '1/2', '1/2'],
  'update_Description': {}},
 'SM Higgs Selfcouplings': {'Lorentz': ['Scalar',
   'Scalar',
   'Scalar',
   'Scalar'],
  'hypercharge': ['1/2', '1/2', '1/2', '1/2'],
  'update_Description': {}},
 'SM Mu Parameter': {'Lorentz': ['Scalar', 'Scalar'],
  'hypercharge': ['1/2', '1/2'],
  'update_Description': {'OutputName': 'm2SM'}},
 'Up-Yukawa-Coupling': {'Lorentz': ['WeylFermion', 'WeylFermion', 'Scalar'],
  'hypercharge': ['1/2', '1/6', '2/3'],
  'update_Description': {}}}

In [17]:
if PARAMETERS:
    us=update_SARAH_parameters(model='SSDM')
    #us=update_SARAH_parameters(model='SM')
    #kk=us.parse_model_particles()
    #kk=us.update_modelparticles()
    kk=us.parse_model_parameters()
    self=us

In [18]:
if PARAMETERS:
    self.parse_Lagrangian() #Fill self.Lagrangian
    self.Lagrangian_Couplings_Descriptions() # Fill self.Lagrangian_Couplings
    self.get_rotations() #Fill self.rotations
    self.get_couplings() #Fill self.couplings
    self.get_constants() #Fill self.couplings 
    self.update_Lagrangian() #Convert some rotations, copuplings, constants to Lagrangian_Couplings
    self.update_modelparameters() # Fill self.updated_modelparameters

In [19]:
self.Higgs

'H'

In [ ]:
#Standard for Scalar Potential:
# Field 1 - Field 2: type of Coupling
self.Lagrangian_Couplings_Singlet_Scalar={
    'Scalar Singlet: Squared Mass': {
    'Description':'Scalar Singlet Mass',
      'Lorentz': ['Scalar', 'Scalar'],
      'hypercharge': ['0', '0'],
      'update_Description': {
          'Real':True,
          'Dependence': None, 
          'Value':None, 
          'LesHouches':['HDM',1]
      }},
    'Scalar Singlet: Selfcoupling': {
      'Lorentz': ['Scalar', 'Scalar', 'Scalar', 'Scalar'],
      'hypercharge': ['0', '0', '0', '0'],
      'update_Description': {
          'LesHouches':['HDM',3]
          }},
    'Scalar Singlet - Higgs: quartic couplings': {
      'Lorentz': ['Scalar', 'Scalar', 'Scalar', 'Scalar'],
      'hypercharge': ['0', '0', '1/2', '1/2'],
      'update_Description': {
          'LesHouches':['HDM',2]
      }}
}

In [13]:
self.Lagrangian_Couplings_Singlet_Scalar

AttributeError: 'update_SARAH_particles' object has no attribute 'Lagrangian_Couplings_Singlet_Scalar'

In [ ]:
len([h for h in [None,None] if h])

In [ ]:
hh=[h for h in self.updated_modelparameters.get('Properties').get('Higgs') if h]
if len(hh)>0:
    Higgs=hh[0]
else:
    Higgs=None

In [ ]:
"""
Strategy:
From sorted quality extract the symbol ->`k` for the coupling Definition (smdict below):
      'Coupling':'Symbol'
Get Higgs -> H From the SM part
From  some selfcoupling extract the BSM Field: Scalar Singlet -> S
If Description include the semicolon separation, define the several Fields
  Example: 'Scalar Singlet - Higgs: quartic couplings'.split(':')[0].split('-')...strip
                 S             H
       Add -> 'Scalar Singlet':'S', 'Higgs': 'H'
"""
dd=self.Lagrangian
smdict=self.Lagrangian_Couplings_Singlet_Scalar
Higgs='H'
sclr_prtcls={'Higgs':Higgs}
#First loop to identify the particles from the Squared Mass couplings
for ds in smdict.keys():
    for k in dd.keys():
        if smdict[ds].get('Lorentz'
         ) and (sorted_equality( smdict[ds]['Lorentz'],dd[k]['Lorentz'] ) and
                sorted_equality( smdict[ds]['hypercharge'],dd[k]['hypercharge'] ) ):
            ll=[l.strip() for l in ds.split(':')]
            prtcls=[l.strip() for l in ll[0].split('-')]
            if len(ll)>1:
                cplng=ll[1]
            if cplng=='Squared Mass':
                sclr_prtcls[prtcls[0]]=dd[k].get('fields')[0]
            print("***",ds,'::',k)

In [ ]:
sclr_prtcls

In [ ]:
for ds in smdict.keys():            ll=[l.strip() for l in ds.split(':')]
            prtcls=[l.strip() for l in ll[0].split('-')]
            if len(ll)>1:
                cplng=ll[1]

    for k in dd.keys():
        if smdict[ds].get('Lorentz'
         ) and (sorted_equality( smdict[ds]['Lorentz'],dd[k]['Lorentz'] ) and
                sorted_equality( smdict[ds]['hypercharge'],dd[k]['hypercharge'] ) ):
            kk=k
            if re.search('^Lam[^bda]',k):
                kk=k.replace('Lam','lambda:')
            print("***",ds,'::',k,symbol_to_OutputName(k),symbol_to_TeX(kk))
            smdict[ds]['Coupling']=k
            # Get Lagrangian data
            for q in dd[k]:
                smdict[ds][q]=dd[k][q]
            #Include particle names
            for p in sclr_prtcls:
                if p in prtcls:
                    smdict[ds][p]=sclr_prtcls[p]
            # Add 'OutputName' and 'LaTeX'
            if smdict[ds].get('update_Description'):
                smdict[ds]['update_Description'][
                    'OutputName']=symbol_to_OutputName(k)
                smdict[ds]['update_Description'][
                    'LaTeX']=symbol_to_TeX(kk)
                
            # if  Higgs
            #print("==",self.Lagrangian_Couplings.get(
            #    'SM Higgs Selfcouplings').get('Higgs') )            

In [ ]:
smdict

In [ ]:
parameters=[]
append_Lagrangian_to_parameters(smdict,parameters)

In [ ]:
self.updated_modelparameters.apply_filter(lambda d:
                                          isinstance(d.get('Properties').get('Higgs'),str) 
                                         )[0].get('Properties').get('Higgs')

In [ ]:
smdict[ds]

In [ ]:
self.updated_modelparameters

In [ ]:
self.updated_modelparticles.loc['"""H']

In [ ]:
1+1

In [ ]:
self.updated_modelparticles.loc['S']

In [ ]:
self.update_Lagrangian??

In [ ]:
self.parse_Lagrangian()

In [ ]:
self.Lagrangian_Couplings_Descriptions()

In [ ]:
{'Scalar Singlet Squared Mass': {
  'Lorentz': ['Scalar', 'Scalar'],
  'hypercharge': ['0', '0'],
  'update_Description': {}},
  'Scalar Singlet Selfcouplings': {
  'Lorentz': ['Scalar', 'Scalar', 'Scalar', 'Scalar'],
  'hypercharge': ['0', '0', '0', '0'],
  'update_Description': {}},
'Scalar Singlet Higgs quartic couplings': {
  'Lorentz': ['Scalar', 'Scalar', 'Scalar', 'Scalar'],
  'hypercharge': ['0', '0', '1/2', '1/2'],
  'update_Description': {}}
}

In [ ]:
{'Scalar Singlet Squared Mass': {'Coupling': 'MS2',
  'Scalar Singlet': 'S',
  'Lorentz': ['Scalar', 'Scalar'],
  'fields': ['S', 'S'],
  'hypercharge': ['0', '0'],
  'update_Description': {}},
  'Scalar Singlet Selfcouplings': {'Coupling': 'LamS',
  'Scalar Singlet': 'S',
  'Lorentz': ['Scalar', 'Scalar', 'Scalar', 'Scalar'],
  'fields': ['S', 'S', 'S', '
             0S'],
  'hypercharge': ['0', '0', '0', '0'],
  'update_Description': {}},
'Scalar Singlet Higgs quartic couplings': {'Coupling': 'LamSH',
  'Scalar Singlet': 'S',
  'Higgs': 'H',
  'Lorentz': ['Scalar', 'Scalar', 'Scalar', 'Scalar'],
  'fields': ['S', 'S', 'H', 'H'],
  'hypercharge': ['0', '0', '1/2', '1/2'],
  'update_Description': {}}
},

In [ ]:
self.update_Lagrangian()

In [ ]:
pd.DataFrame(self.update_modelparameters())

In [ ]:
import sys
if PARAMETERS:
    sys.exit('Stop')

## SPheno

In [ ]:
class update_SARAH(update_SARAH_parameters):
    def __init__(self,*args, **kwargs):
        self.SPheno={'Properties':{
         'OnlyLowEnergySPheno' : True,
         'AddTreeLevelUnitarityLimits':True,
        'MINPAR':[],
        'ParametersToSolveTadpoles':[],
        'BoundaryLowScaleInput':[],
        'ListDecayParticles':[],
        'ListDecayParticles3B': [],
        'RenConditionsDecays':[
               ['dCosTW', '1/2*Cos[ThetaW] * (PiVWp/(MVWp^2) - PiVZ/(mVZ^2))'],
               ['dSinTW', '-dCosTW/Tan[ThetaW]'],
               ['dg2', '1/2*g2*(derPiVPheavy0 + PiVPlightMZ/MVZ^2 - (-(PiVWp/MVWp^2) + PiVZ/MVZ^2)/Tan[ThetaW]^2 + (2*PiVZVP*Tan[ThetaW])/MVZ^2)'],
               ['dg1', 'dg2*Tan[ThetaW]+g2*dSinTW/Cos[ThetaW]- dCosTW*g2*Tan[ThetaW]/Cos[ThetaW]']
             ],
        'DEFINITION': {'MatchingConditions':
                 []},
        'DefaultInputValues' :{}
           },
        'Index':{'OnlyLowEnergySPheno':0,'MINPAR':1, 'ParametersToSolveTadpoles':2, 
                'BoundaryLowScaleInput':3, 'DEFINITION':4, 'ListDecayParticles':5, 
                'ListDecayParticles3B':6, 'DefaultInputValues':7, 
                'AddTreeLevelUnitarityLimits':8, 'RenConditionsDecays':9}   
        }
        super().__init__(*args,**kwargs)
    
    def update_SPheno(self):
        parameters=self.updated_modelparameters
        sci=extract_Lagrangian_terms(parameters,
                                     Lagrangian_dimension=4,
                                     Lorentz_structures=1).get('Symbol')

        sciIN=get_input_parameters_IN(sci,suffix='IN')

        self.SPheno['Properties'
                   ]['BoundaryLowScaleInput']=get_BoundaryLowScaleInput(sci,sciIN)

        self.SPheno['Properties'
                   ]['MINPAR']=get_MINPAR(sciIN)

        self.SPheno['Properties'
                   ]['ParametersToSolveTadpoles'
                    ]=get_tadpoles_and_bilinears(parameters,exclude=[])[0]

        self.SPheno['Properties'][
            'DEFINITION']['MatchingConditions']=get_SM_MatchingConditions(parameters)

        DP,DP3B=get_decay_particles(DecayParticles   = ['Fu', 'Fe', 'Fd', 'hh'],
                            DecayParticles3B = ['Fu', 'Fe', 'Fd'])
        self.SPheno['Properties'
                   ]['ListDecayParticles']=DP
        self.SPheno['Properties'
                   ]['ListDecayParticles3B']=DP3B

        self.SPheno['Properties'][
            'DefaultInputValues'].update( 
                             get_sm_DefaultInputValues(parameters,sci,sciIN) )
        return self.SPheno
    
    def update_SARAH_full(self):
        kk=self.parse_model_parameters()
        kk=self.update_SPheno()

In [ ]:
if SPHENO:
    us=update_SARAH(model='SSDM')
    kk=us.update_SARAH_full()

In [ ]:
if SPHENO:
    pd.DataFrame(us.SPheno)

# Output

In [ ]:
class SARAH_output(update_SARAH):
    def __init__(self,*args, **kwargs):
        self.output_dir=kwargs.get('output_dir')
        self.PARSE=False
        super().__init__(*args,**kwargs)

    def check_output_dir(self):
        if not self.output_dir:
            self.output_dir='{}{}'.format(
                                self.model_dir,
                                self.model_name
                                        ).strip()

        if not re.search('\/$',self.output_dir):
            self.output_dir='{}/'.format(self.output_dir)

        if not os.path.exists(self.output_dir):
            os.mkdir( self.output_dir )
            
    def parse_SARAH_full():
        self.PARSE=True
        kk=self.update_SARAH_full()


    def to_math(self,particles_name ='particles.m',
                     parameters_name='parameters.m',
                     SPheno_name    ='SPheno.m'):
        if not self.PARSE:
            kk=self.parse_SARAH_full()
        kk=self.check_output_dir()
        # Prepare output files
        dirpath='{}{}'.format(self.output_dir,self.model_name)
        if not re.search('\/$',dirpath):
            dirpath='{}/'.format(dirpath)
        if not os.path.isdir(dirpath):
            os.mkdir(dirpath)
        model_path     ='{}{}'.format(dirpath,self.model_file_name)
        particles_path ='{}{}'.format(dirpath,particles_name)
        parameters_path='{}{}'.format(dirpath,parameters_name)
        SPheno_path    ='{}{}'.format(dirpath,SPheno_name)

        #write output files
        WRITE=True
        if os.path.isfile(model_path):
            CHKW=input('File {} exists.Overwrite? (y/n)'.format(
                                  model_path ))
            if re.search('^\s*[nN][oOtT]{0,2}',CHKW):
                WRITE=False
        if WRITE:
            f=open(model_path,'w')
            f.write(self.model_file)
            f.close()
            
        PDF=to_defintions(self.updated_modelparticles,symbol='Field')
        dfnp=pd.DataFrame(PDF)
        kk=to_math(dfnp,particles_path)
        PDF=to_defintions(self.updated_modelparameters,symbol='Symbol')
        dfnp=pd.DataFrame(PDF)
        kk=to_math(dfnp,parameters_path,definitions='ParameterDefinitions')
        SP=pd.DataFrame(self.SPheno)
        kk=to_SPheno(SP,SPheno_path,dictentries=['DefaultInputValues'])        

In [ ]:
if OUTPUT:
    us=SARAH_output(model='SSDM',output_dir='tmp')
    kk=us.to_math()

In [ ]:
def get_inert_scalars(p,particles,
                                      pids={'PDG':[6666635],
                                           'PDG.IX':[101000002],
                                           'FeynArtsNr':[10]
                                           }):
    #INERT scalars
    kk=particles.loc[p.get('Parents')]
    #singlet real scalar
    if kk.get('Properties'
        ) and get_abs_groups( kk.get('Properties').get('Groups')[0],
                               u1_abs='0',su2='1',su3_abs='1'):
        p['Definition']='EWSB'
        p['Description']='Singlet Inert Scalar'
        #TODO: Get from fullparticlesnames.json
        if not p.get('Properties'):
            p['Properties']={}
        p['Properties']['update_Description']= {'PDG' : [pids.get('PDG')[0]],
                                  'PDG.IX' : [pids.get('PDG.IX')[0]],
                                  'FeynArtsNr' : pids.get('FeynArtsNr')[0],
                                  'Mass': 'LesHouches',               
                                  'LaTeX' : 'S',
                                  'ElectricCharge' : 0,
                                  'LHPC' : ['gold'],
                                  'OutputName' : 'Ss'
                                 }
        return p
    
class bsm(SARAH_output):
    def __init__(self,*args, **kwargs):
        self.output_dir=kwargs.get('output_dir')
        super().__init__(*args,**kwargs)
    def parse_bsm(self):
        self.PARSE=True
        kk=self.update_SARAH_full()
        #New particle properties
        kk=self.update_particles(get_inert_scalars,Definition='GaugeES',
                                    Lorentz='Scalar',
                                    args=(self.modelparticles,
                                          {'PDG':[6666635],
                                           'PDG.IX':[101000002],
                                           'FeynArtsNr':[10]
                                          }
                                     ))

In [ ]:
if BSM:
    us=bsm(model='SSDM',output_dir='tmp')
    kk=us.parse_bsm()
    self=us
    kk=us.to_math()

In [ ]:
a=x

In [ ]:
self.update_particles(get_inert_scalars)

In [ ]:
self.updated_modelparticles.loc['ss']

In [ ]:
self.updated_modelparticles.loc['hh']

In [ ]:
self.update_particles(get_inert_scalars,Definition='GaugeES',
                                    Lorentz='Scalar',
                                    args=(self.modelparticles,
                                          {'PDG':[6666635],
                                           'PDG.IX':[101000002],
                                           'FeynArtsNr':[10]
                                          }
                                     ))